In [1]:
import io
from PIL import Image
from pathlib import Path
import os
import torch
from sklearn.manifold import TSNE
import pickle
import numpy as np
import matplotlib.pyplot as plt
os.environ['HF_HOME'] = '/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.cache' ## THIS HAS TO BE BEFORE YOU IMPORT TRANSFORMERS
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def setup_model() -> tuple:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float16

    processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
    generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")
    model = AutoModelForCausalLM.from_pretrained(
        "StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True
    ).to(device)

    return processor, model, device, dtype, generation_config

processor, model, device, dtype, generation_config = setup_model()

Loading checkpoint shards: 100%|██████████| 7/7 [01:41<00:00, 14.56s/it]


In [3]:
patch_embeddings_outputs = []
post_layer_norm_outputs = []
q_former_outputs = []
language_projection_outputs = []
mistral_model_outputs = []

patch_embeddings_output = None
post_layer_norm_output = None
q_former_output = None
language_projection_output = None
mistral_model_output = None

def patch_embedding_hook(module, input, output):
    """Function to be called by the hook for the patch embeddings layer."""
    output = output.cpu().detach()  # Assuming you want to move data to CPU for analysis
    
    global patch_embeddings_output 
    patch_embeddings_output = output
    
    # patch_embeddings_outputs.append(output)  

def post_layer_norm_hook(module, input, output):
    """Function to be called by the hook for the post layer norm layer."""
    output = output.cpu().detach()  # Assuming you want to move data to CPU for analysis
    
    global post_layer_norm_output
    post_layer_norm_output = output
    
    # post_layer_norm_outputs.append(output) 

def language_projection_hook(module, input, output):
    """Function to be called by the hook for the language projection layer."""
    input = input[0].cpu().detach()  # Assuming you want to move data to CPU for analysis
    output = output.cpu().detach()  # Assuming you want to move data to CPU for analysis
    
    global q_former_output
    q_former_output = input

    global language_projection_output
    language_projection_output = output

    # q_former_outputs.append(input)
    # language_projection_outputs.append(output)

def mistral_model_output_hook(module, input, output):
    """Function to be called by the hook for the mistral norm layer."""
    output = output.cpu().detach()  # Assuming you want to move data to CPU for analysis
    
    global mistral_model_output
    mistral_model_output = output

    # mistral_model_outputs.append(output)


In [4]:
embeddings_dict = {}
def generate_with_forward_hooks(images, prompt, processor, model, device, dtype, generation_config):
    image_id = images
    images = Image.open(image_id).convert("RGB")

    # convert image_id to a string
    if isinstance(image_id, Path):
        image_id_string = str(image_id).split("/")[-1].split(".")[0]

    # register hooks
    patch_embeddings = model.vision_model.embeddings.patch_embedding.register_forward_hook(patch_embedding_hook)
    post_layer_norm = model.vision_model.post_layernorm.register_forward_hook(post_layer_norm_hook)
    language_projection = model.language_projection.register_forward_hook(language_projection_hook)
    # mistral_model = model.language_model.model.norm.register_forward_hook(mistral_model_output_hook)

    # complete a forward pass 
    inputs = processor(
        images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt"
    ).to(device=device, dtype=dtype)
    output = model.generate(**inputs, generation_config=generation_config)[0]
    response = processor.tokenizer.decode(output, skip_special_tokens=True)

    global embeddings_dict
    embeddings_dict[image_id_string] = {
        'patch_embeddings': patch_embeddings_output.cpu().numpy(),
        'post_layer_norm': post_layer_norm_output.cpu().numpy(),
        'q_former': q_former_output.cpu().numpy(),
        'language_projection': language_projection_output.cpu().numpy(),
    }

    # remove hooks
    patch_embeddings.remove()
    post_layer_norm.remove()
    language_projection.remove()
    # mistral_model.remove()

    return response

prompt = "Describe the findings"


In [6]:
vindr_dir = Path('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/')
# list all images in the directory
images = list(vindr_dir.glob('*.png'))
# image = Image.open(image).convert("RGB")
print(images[:5])

[PosixPath('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/f3426b51acbf433d03c5f84a5d16c0d3.png'), PosixPath('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/0b877b934aada3284ac13cfa74b5419c.png'), PosixPath('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/235fa5309a20f02f306297fee158a109.png'), PosixPath('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/6039745a7cf89b4f65da84eaa8c1a226.png'), PosixPath('/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/0a8dc3de200bc767169b73a6ab91e948.png')]


In [84]:
for image in images[:3]:
    response = generate_with_forward_hooks(image, prompt, processor, model, device, dtype, generation_config)
    print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


There is no pneumothorax or pleural effusion. There is no focal consolidation or pulmonary edema. The cardiomediastinal silhouette is within normal limits.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The heart size is normal. The mediastinal and hilar contours are normal. The pulmonary vasculature is normal. The lungs are clear. No pleural effusion or pneumothorax is seen.
The lungs are hyperinflated with flattening of the diaphragms. There is no focal consolidation. There is no pleural effusion or pneumothorax. The cardiomediastinal silhouette is within normal limits.
[]


In [118]:
import pickle

In [ ]:
# Define a function to save and clear the global lists to manage memory
embeddings_dict = {}
file_path = Path('/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired')
file_path.mkdir(parents=True, exist_ok=True)

def save_embeddings_dict_pickle(embeddings_dict, batch_id, file_path):
    file_path.mkdir(parents=True, exist_ok=True)
    with open(file_path / f'embeddings_batch_{batch_id}.pkl', 'wb') as f:
        pickle.dump(embeddings_dict, f)
    embeddings_dict.clear()  # Clear the dictionary after saving

batch_size = 100  # Choose an appropriate size
num_images = len(images)
num_batches = (num_images + batch_size - 1) // batch_size  # Ceiling division to account for the last batch

for batch_id in range(num_batches):
    batch_start = batch_id * batch_size
    batch_end = min(batch_start + batch_size, num_images)
    batch_images = images[batch_start:batch_end]
    # generate the list of file names coreesponding to the images
    file_names = [image.name for image in batch_images]
    print(file_names)
    for image in batch_images:
        response = generate_with_forward_hooks(image, prompt, processor, model, device, dtype, generation_config)
    
    # After processing a batch, save the hook outputs and clear them from memory
    save_embeddings_dict_pickle(embeddings_dict, batch_id, file_path)
    # save_and_clear_hooks_output(batch_id,file_names)


In [ ]:
print(embeddings_dict)
print(type(embeddings_dict['f3426b51acbf433d03c5f84a5d16c0d3']))   

In [ ]:
import random
# compare the values of the embeddings in the pickle file and the embeddings_dict
file_path = Path('/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired')
file_path.mkdir(parents=True, exist_ok=True)
for i in range(30):
    print(f'Batch {i}')
    with open(file_path / f'embeddings_batch_{i}.pkl', 'rb') as f:
        embeddings_dict_pickled = pickle.load(f)

        embeddings_dict_pickled_keys = embeddings_dict_pickled.keys()
        # randomly select 10 keys from the embeddings_dict_pickled_keys and compare the values with generated with forward hooks
        n = 25
        random_keys = random.sample(embeddings_dict_pickled_keys, n)

        for key in random_keys:
            image = vindr_dir / f'{key}.png'
            response = generate_with_forward_hooks(image, prompt, processor, model, device, dtype, generation_config)
            if not np.allclose(embeddings_dict_pickled[key]['patch_embeddings'], patch_embeddings_output.cpu().numpy()):
                print(f'Error in patch embeddings for {key}')
            if not np.allclose(embeddings_dict_pickled[key]['post_layer_norm'], post_layer_norm_output.cpu().numpy()):
                print(f'Error in post layer norm embeddings for {key}')
            if not np.allclose(embeddings_dict_pickled[key]['q_former'], q_former_output.cpu().numpy()):
                print(f'Error in q former embeddings for {key}')
            if not np.allclose(embeddings_dict_pickled[key]['language_projection'], language_projection_output.cpu().numpy()):
                print(f'Error in language projection embeddings for {key}')

In [69]:
batch = 0

# open the saved embeddings
batch_patch_embeddings = torch.load(f'/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired/patch_embeddings_batch_{batch}.pt')
batch_post_layer_norm_outputs = torch.load(f'/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired/post_layer_norm_outputs_batch_{batch}.pt')
batch_q_former_outputs = torch.load(f'/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired/q_former_outputs_batch_{batch}.pt')
batch_language_projection_outputs = torch.load(f'/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test_paired/language_projection_outputs_batch_{batch}.pt')

In [71]:
print(len(batch_patch_embeddings))

150


In [33]:
with open(f'/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/model_inspection/embeddings/VinDr_test/file_names_batch_{batch_id}.txt', 'r') as f:
    file_names = f.readlines()
    file_names = [file_name.strip() for file_name in file_names]


for index, file in enumerate(file_names):
    # clear the lists
    patch_embeddings_outputs.clear()
    post_layer_norm_outputs.clear()
    q_former_outputs.clear()
    language_projection_outputs.clear()

    image = Image.open(f'/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/{file}').convert("RGB")
    response = generate_with_forward_hooks(image, prompt, processor, model, device, dtype, generation_config)
    # check if the embeddings are the same
    if not torch.allclose(patch_embeddings_outputs[0], batch_patch_embeddings[index]):
        print(f"Patch embeddings are not the same for {file}")
    
    if not torch.allclose(post_layer_norm_outputs[0], batch_post_layer_norm_outputs[index]):
        print(f"Post layer norm outputs are not the same for {file}")
        
    if not torch.allclose(q_former_outputs[0], batch_q_former_outputs[index]):
        print(f"Q former outputs are not the same for {file}")
    
    if not torch.allclose(language_projection_outputs[0], batch_language_projection_outputs[index]):
        print(f"Language projection outputs are not the same for {file}")

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.cache/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Patch embeddings are not the same for f3426b51acbf433d03c5f84a5d16c0d3.png
Post layer norm outputs are not the same for f3426b51acbf433d03c5f84a5d16c0d3.png
Q former outputs are not the same for f3426b51acbf433d03c5f84a5d16c0d3.png
Language projection outputs are not the same for f3426b51acbf433d03c5f84a5d16c0d3.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Patch embeddings are not the same for 0b877b934aada3284ac13cfa74b5419c.png
Post layer norm outputs are not the same for 0b877b934aada3284ac13cfa74b5419c.png
Q former outputs are not the same for 0b877b934aada3284ac13cfa74b5419c.png
Language projection outputs are not the same for 0b877b934aada3284ac13cfa74b5419c.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Patch embeddings are not the same for f5de7e513d2c4085e8b7fd6b5e5db57a.png
Post layer norm outputs are not the same for f5de7e513d2c4085e8b7fd6b5e5db57a.png
Q former outputs are not the same for f5de7e513d2c4085e8b7fd6b5e5db57a.png
Language projection outputs are not the same for f5de7e513d2c4085e8b7fd6b5e5db57a.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Patch embeddings are not the same for c6829051ecb281656c987fa2cfe5c706.png
Post layer norm outputs are not the same for c6829051ecb281656c987fa2cfe5c706.png
Q former outputs are not the same for c6829051ecb281656c987fa2cfe5c706.png
Language projection outputs are not the same for c6829051ecb281656c987fa2cfe5c706.png


KeyboardInterrupt: 

In [15]:
# compare the embeddings
# patch embeddings
print(torch.allclose(batch_0_patch_embeddings[0], patch_embeddings_outputs[0]))
# post layer norm
print(torch.allclose(batch_0_post_layer_norm_outputs[0], post_layer_norm_outputs[0]))
# q_former
print(torch.allclose(batch_0_q_former_outputs[0], q_former_outputs[0]))
# language projection
print(torch.allclose(batch_0_language_projection_outputs[0], language_projection_outputs[0]))

True
True
True
True


In [16]:
print(batch_0_q_former_outputs[0])

tensor([[[-0.2561, -0.2659,  1.2744,  ...,  0.2036, -0.5537, -1.1152],
         [-0.2566, -0.2654,  1.2744,  ...,  0.2036, -0.5542, -1.1143],
         [-0.2559, -0.2646,  1.2734,  ...,  0.2040, -0.5532, -1.1133],
         ...,
         [-0.2563, -0.2651,  1.2744,  ...,  0.2040, -0.5527, -1.1143],
         [-0.2908, -0.2450,  1.2832,  ...,  0.1680, -0.4976, -1.1416],
         [-0.2561, -0.2656,  1.2744,  ...,  0.2040, -0.5537, -1.1152]]],
       dtype=torch.float16)
